In [36]:
import mysql.connector
# library for data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense
from sklearn.model_selection import train_test_split 
from sklearn.metrics.pairwise import cosine_similarity

import os 

In [37]:
# Membuat koneksi ke database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tourista_db'
)

In [38]:
# Mengeksekusi query untuk mengambil data dari tabel

query = "SELECT * FROM destination"
destination = pd.read_sql_query(query, conn)

query = "SELECT * FROM review_wisata"
ratings = pd.read_sql_query(query, conn)

query = "SELECT * FROM user_profile"
users = pd.read_sql_query(query, conn)

C:\Users\dewan\AppData\Local\Temp\ipykernel_11652\707316895.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  destination = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_11652\707316895.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_11652\707316895.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql_query(query, conn)


## Data preparation

In [39]:
destination.head(2)

,id,name_wisata,description_wisata,category,destination_photo,city,price,rating,time_minutes,coordinate,destination_lat,destination_long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,lala.png,Jakarta,20000.0,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.17539,106.8272
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,lala.png,Jakarta,5000.0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.13764,106.8171


In [40]:
ratings.head(2)

,user_wisata,wisata_id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [41]:
users.head()

,id,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
1,2,user2,21,userTrial2@gmail.com,81326812647,"Bekasi, Jawa Barat",user2.png,-11931.0,12311.0,None,None
2,3,user3,23,userTrial3@gmail.com,81326812647,"Cirebon, Jawa Barat",user3.png,-11931.0,12311.0,None,None
3,4,user4,21,userTrial4@gmail.com,81326812647,"Bekasi, Jawa Barat",user4.png,-11931.0,12311.0,None,None
4,5,user5,20,userTrial5@gmail.com,81326812647,"Lampung, Sumatera Selatan",user5.png,-11931.0,12311.0,None,None


## Model

In [42]:
# fungsi untuk melakukan groouping data
def groupingCategory(df, budget, totalCategory, excepts = []):
    data = []
    
    if len(excepts) == 0:
        for idx, row in df.iterrows():
            if len(data) == totalCategory:
                break
            if row['price'] < budget:
                data.append(row['name_wisata'])
                budget -= row['price']    
    else:
        for x in excepts:
            if df['name_wisata'].eq(x).any():
                df = df.loc[df['price'] != x]
                
        for idx, row in df.iterrows():
            if len(data) == totalCategory:
                break
            if row['price'] < budget:
                data.append(row['name_wisata'])
                budget -= row['price']
        
    return data

In [43]:
# Function to recommend places based on user input
def recommend_places(df, category, city, price, rating, top_n=50):
    # Filter dataset based on user input
    filtered_df = df[(df['category'] == category) & (df['city'] == city) & (df['price'] <= price) & (df['rating'] >= rating)]
    
    # Calculate cosine similarity between user input and dataset
    user_input = [[price, rating]]
    dataset = filtered_df[['price', 'rating']]
    similarity_matrix = cosine_similarity(user_input, dataset)
    
    # Sort places based on similarity score
    filtered_df['Similarity'] = similarity_matrix[0]
    recommended_places = filtered_df.sort_values(by='Similarity', ascending=False).head(top_n)
    
    gold = []
    silver = []
    bronze = []
    
    gold = groupingCategory(recommended_places, price, 5)
    silver = groupingCategory(recommended_places, price, 5, gold)
    bronze = groupingCategory(recommended_places, price, 5, (silver + gold))
                
    return {"gold" : gold,
            "silver" : silver, 
            "bronze" : bronze}

## code example

In [44]:
# Example usage
category = 'Taman Hiburan'
city = 'Jakarta'
price = 50000
rating = 4.0

In [45]:
recommendations = recommend_places(destination, category, city, price, rating)
recommendations

C:\Users\dewan\AppData\Local\Temp\ipykernel_11652\2665731653.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Similarity'] = similarity_matrix[0]


{'gold': ['Wisata Agro Edukatif Istana Susu Cibugary',
  'Jakarta Planetarium',
  'Hutan Kota Srengseng',
  'Taman Menteng',
  'Taman Suropati'],
 'silver': ['Wisata Agro Edukatif Istana Susu Cibugary',
  'Jakarta Planetarium',
  'Hutan Kota Srengseng',
  'Taman Menteng',
  'Taman Suropati'],
 'bronze': ['Wisata Agro Edukatif Istana Susu Cibugary',
  'Jakarta Planetarium',
  'Hutan Kota Srengseng',
  'Taman Menteng',
  'Taman Suropati']}

## update database

In [46]:
cursor = conn.cursor()

In [47]:
user_id = 1

In [48]:
sql = "INSERT INTO trip_detail (user_id , trip_name_type, name_wisata) VALUES (%s, %s, %s)"

In [49]:
for category, places in recommendations.items():
    for place in places:
        values = (user_id, category, place)
        cursor.execute(sql, values)

In [51]:
conn.commit()